# Gradio custom Bot with LangChain ConversationEntityMemory

https://www.linkedin.com/pulse/build-qa-bot-over-private-data-openai-langchain-leo-wang/

In [1]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains.llm import LLMChain
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationEntityMemory
from langchain.chains.conversation.prompt import ENTITY_MEMORY_CONVERSATION_TEMPLATE

import os
os.environ["OPENAI_API_KEY"] = ""

#llm = ChatOpenAI(temperature=0, model='gpt-3.5-turbo')
#llm = OpenAI(temperature=0, model='gpt-3.5-turbo')

## Define Chain

In [3]:
# Define the LLM chat model
llm = ChatOpenAI(temperature=0.9)

# Create a ConversationEntityMemory object if not already created
memory = ConversationEntityMemory(llm=llm, k=10)
# combine them into a chain
conversation = ConversationChain(
    llm=llm,
    prompt=ENTITY_MEMORY_CONVERSATION_TEMPLATE,
    memory=memory,
    verbose=True,
)

Test the Chain:

In [ ]:
# output = conversation.predict(
#     input="""
#     John helped Max study for a math exam. 
#     Thanks to him, Max got a high grade from the exam.
#     """)
# print(output)

Here is the logic:

1. Start a new variable "chat_history" with empty string
2. Always pass the user question and history to the model
3. Append the answer to the chat history
4. Repeat

It is literally three lines of code. I had a function only because of the front end.

In [ ]:
# Front end web app
import gradio as gr
demo = gr.Blocks()
with demo:
    gr.Markdown(
        """
        # 🦜🔗 Test ChatBot with EntityMemory
        """
    )
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.Button("Clear")
    
    def user(user_message, history):
        # Format the list according to the expected input by ConversationalRetrievalChain
        history = [(item[0], item[1]) for item in history]
        # Get response from QA chain (history not used here, it is already buffered)
        response = conversation.run(user_message)
        # Keep the same ouput as before avoid error in Gradio, but explicit history is not used in QA chain
        history.append((user_message, response))
        return gr.update(value=""), history
    
    msg.submit(user, inputs=[msg, chatbot], outputs=[msg, chatbot], queue=False)
    clear.click(lambda: None, None, chatbot, queue=False)

demo.launch(debug=True)

Now with a custom bot and streaming the bot output

In [5]:
import gradio as gr
import random
import time
# Add presets for Gradio theme
from app_modules.presets import * 
# Add custom CSS
with open("assets/custom.css", "r", encoding="utf-8") as f:
    customCSS = f.read()

with gr.Blocks(css=customCSS, theme=small_and_beautiful_theme) as demo:
    
    gr.Markdown(
        """
        # 🦜🔗 Test Gradio ChatBot with LangChain EntityMemory
        """
    )
    
    # Start chatbot with welcome from bot
    chatbot = gr.Chatbot([(None,'How can I help you?')]).style(height=650)
    msg = gr.Textbox()
    clear = gr.ClearButton([msg, chatbot])

    def user(user_message, history):
        return gr.update(value="", interactive=False), history + [[user_message, None]]

    def bot(history):
        user_message = history[-1][0] # get if from most recent history element
        bot_message  = conversation.run(user_message)
        history[-1][1] = ""
        for character in bot_message:
            history[-1][1] += character
            #time.sleep(0.05)
            yield history

    response = msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, chatbot, chatbot
    )
    response.then(lambda: gr.update(interactive=True), None, [msg], queue=False)

demo.queue()
demo.launch(debug=True)

/Users/inigo/.local/share/virtualenvs/auto-agent-npQlq64W/lib/python3.10/site-packages/gradio/components/chatbot.py:228: UserWarning: The `style` method is deprecated. Please set these arguments in the constructor instead.
  warnings.warn(


Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.




> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised ServiceUnavailableError: The server is overloaded or not ready yet..




> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

In [23]:
conversation.memory.entity_store.store

{'John': 'John helped Max study for a math exam and as a result, Max got a high grade from the exam.',
 'Max': "Based on the information provided, I know that Max is graduating from college this semester and has two children to take care of. It seems like he's juggling a lot of responsibilities, but is still working hard to achieve his goals."}

In [30]:
conversation.memory.



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

In [29]:
conversation.memory.entity_store.store

{'Max': 'Max has two children that he needs to take care of.',
 'John': 'John helped Max study for a math exam and as a result, Max received a high grade from the exam.'}



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 